In [1]:
import matplotlib.pyplot as plt
import hyperspy.api as hs 
import numpy as np
import matplotlib.image as mpimg
import scipy.ndimage as ndim

%matplotlib qt5

In [2]:
# spook = hs.load("example_data/stem_dpc_data.zspy", lazy=True)
s = hs.load("our_data/LMSTEM_256x256pix_15x15step_2msExpTime_250msFlyback_regular.zspy", lazy=True)

In [3]:
s_sum = s.sum(axis=(-1, -2))
s_nav = s_sum.transpose()
s_nav.compute()
s.navigator = s_nav
s_nav

[########################################] | 100% Completed | 12.37 s


<ElectronDiffraction2D, title: , dimensions: (|256, 256)>

In [14]:
# s_nav.plot()

In [5]:
# cx, cy = (122, 116) # Eksempeldata
cx, cy = (120, 127) # bilde av regulaer
smigel = s.isig[cx-50:cx+50, cy-50:cy+50]
smigel.navigator = s_nav
smigel.compute()

[########################################] | 100% Completed | 3.08 sms


In [49]:
smigel.T.plot()

## Mer avansert analyse

En litt mer avansert måte å analysere dette, er å bruk `center_of_mass` funksjonen i `s1`. Lagre dette som `s1_com`. Dette regner ut hvor senter-posisjonen er for alle probe-posisjonene.

In [6]:
com = smigel.center_of_mass()
coomer = com.correct_ramp()

[########################################] | 100% Completed | 6.61 sms


In [10]:
# Blur
coomer_blur = coomer.gaussian_blur(1.2)
coomer_blur.get_color_signal(autolim_sigma=4).plot()

Så plot `s1_com_corr`, ved å bruke `get_color_signal().plot()`.

In [56]:
coomer.get_color_signal().plot()

In [169]:
print(coomer)

<DPCSignal2D, title: , dimensions: (2|256, 256)>


In [7]:
mask = mpimg.imread("hestesko_regulaer.png")
mask = mask[:, :, 0]
mask = np.array(mask, dtype=bool)
# mask = np.logical_not(mask)

In [114]:
plt.imshow(mask, cmap="binary")
plt.show()

In [175]:
fig, ax = plt.subplots()
mcp = mask.copy()
mcp = ndim.rotate(mcp, -45, order=0)
ax.imshow(mcp, cmap="binary")
fig.show()

In [8]:
masked = coomer.deepcopy()
mcp = mask.copy()
fromx = 940
fromy = 900
rotation = 130
zoom = 0.944
masky = ndim.rotate(mcp, rotation, order=0)
masky = ndim.zoom(masky, zoom, order=0)
masky = masky[fromx:fromx+256*4:4, fromy:fromy+256*4:4]
masked.data[:, masky] *= 0.0
# masked.get_color_signal().plot()

In [268]:
# masked.calibrate()
# coomer.calibrate()

## Plotting av disse dataene

Nå kan dette kombineres med kunnskapen og koden dere brukte i TEM-bildedata notebooken, og FIB notebooken, til å lage en figur som ligner på den i starten av denne Jupyter Notebooken.

For å få disse fargeplottene i en matplotlib-figur, så først lag en `fig` og en `ax` via matplotlib.

Så bruk `get_color_image_with_indicator` i `s1_com_corr` med `ax=ax`.

In [9]:
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as patheffects

fontprops = fm.FontProperties(size=18)

scalebar_kwargs = {'size': 2, 'label': '2 $\mathrm{\mu}$m', 'loc': 4, 'frameon': False, 'color': 'white', 'size_vertical': 0.2, 'label_top': False, 'fontproperties': fontprops}
nice_effect = [patheffects.withStroke(linewidth=2, foreground='black', capstyle="round")]

def add_scalebar(ax: plt.Axes): 
    scalebar = AnchoredSizeBar(transform=ax.transData, **scalebar_kwargs)
    # Denne legger til et svart omriss rundt scalebar teksten, for å gjøre den lettere å lese
    scalebar.txt_label._text.set_path_effects(nice_effect)
    ax.add_artist(scalebar)

In [11]:
fig, (ax, ax2) = plt.subplots(1, 2)
inpush = ax.inset_axes([0.83, 0.83, 0.15, 0.15])
inpush2 = ax2.inset_axes([0.83, 0.83, 0.15, 0.15])
coomer.get_color_image_with_indicator(ax=ax, ax_indicator=inpush, autolim_sigma=2, indicator_rotation=90)
masked.get_color_image_with_indicator(ax=ax2, ax_indicator=inpush2, indicator_rotation=90)
ax.axis("off")
ax2.axis("off")
add_scalebar(ax)
add_scalebar(ax2)
ax.annotate("a",xy=(0.05, 0.90), xycoords="axes fraction",fontsize=20, color="w", path_effects=nice_effect)
ax2.annotate("b",xy=(0.05, 0.90), xycoords="axes fraction",fontsize=20, color="w", path_effects=nice_effect)
fig.tight_layout()
fig.show()

Tips: posisjonen til fargehjulet kan styres med `ax_indicator` parameteren, se docstringen for mer informasjon.

In [12]:
fig.savefig("dpc_regulaer.png", dpi=300, bbox_inches="tight", pad_inches=0, transparent=True)